# 🎨 Noobai-XL-1.1 Text-to-Image Generator

### Функціонал (Features):
- Різні методи семплінгу (Various sampling methods)
- Налаштування кроків (Adjustable sampling steps)
- Контроль seed (Seed control)
- Налаштування CFG Scale
- HiRes Fix з 8x-NKMD-Superscale
- Налаштування розміру зображення (Image size control)
- Без обмеження токенів (No token limit)

In [ ]:
!pip install diffusers==0.25.0 transformers==4.36.0 torch==2.1.0 accelerate
!pip install xformers
!pip install opencv-python-headless
!pip install gradio

In [ ]:
import torch
from diffusers import DiffusionPipeline, AutoencoderKL, DDIMScheduler, DPMSolverMultistepScheduler
from transformers import CLIPTokenizer
import cv2
import numpy as np
import gradio as gr
from PIL import Image
import random

In [ ]:
# Завантаження моделі (Model loading)
def load_model():
    model_id = "Laxhar/noobai-XL-1.1"
    
    # Налаштування токенайзера без лімітів (Setting up tokenizer without limits)
    tokenizer = CLIPTokenizer.from_pretrained(
        model_id,
        subfolder="tokenizer",
        use_fast=False,
        model_max_length=512  # Збільшений ліміт токенів
    )

    # Завантаження основної моделі (Loading main model)
    pipe = DiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )
    
    pipe = pipe.to("cuda")
    pipe.enable_xformers_memory_efficient_attention()
    
    return pipe

# Функція для HiRes Fix (HiRes Fix function)
def apply_hires_fix(image, scale_factor, creativity_steps):
    img_np = np.array(image)
    height, width = img_np.shape[:2]
    new_height = int(height * scale_factor)
    new_width = int(width * scale_factor)
    
    # 8x-NKMD-Superscale implementation
    for _ in range(creativity_steps):
        img_np = cv2.resize(img_np, (new_width, new_height), 
                           interpolation=cv2.INTER_LANCZOS4)
        # Advanced sharpening
        kernel = np.array([[0, -1, 0],
                          [-1, 5, -1],
                          [0, -1, 0]])
        img_np = cv2.filter2D(img_np, -1, kernel)
    
    return Image.fromarray(img_np)

# Головна функція генерації (Main generation function)
def generate_image(prompt, negative_prompt, sampler, steps, seed, cfg_scale, 
                  width, height, use_hires, hires_scale, hires_steps):
    if seed == -1:
        seed = random.randint(0, 2147483647)
    
    # Вибір семплера (Sampler selection)
    if sampler == "DDIM":
        pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
    elif sampler == "DPM++ 2M":
        pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    
    # Генерація зображення (Image generation)
    generator = torch.Generator("cuda").manual_seed(seed)
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=steps,
        generator=generator,
        guidance_scale=cfg_scale,
        width=width,
        height=height
    ).images[0]
    
    # Застосування HiRes Fix якщо вибрано (Apply HiRes Fix if selected)
    if use_hires:
        image = apply_hires_fix(image, hires_scale, hires_steps)
    
    return image, seed

# Створення інтерфейсу Gradio (Creating Gradio interface)
def create_interface():
    with gr.Blocks() as interface:
        with gr.Row():
            with gr.Column():
                prompt = gr.Textbox(label="Prompt", lines=3)
                negative_prompt = gr.Textbox(label="Negative Prompt", lines=3)
                
                with gr.Row():
                    sampler = gr.Dropdown(["DDIM", "DPM++ 2M"], label="Sampler", value="DDIM")
                    steps = gr.Slider(1, 100, value=30, label="Steps")
                    
                with gr.Row():
                    seed = gr.Number(label="Seed (-1 for random)", value=-1)
                    cfg_scale = gr.Slider(1, 20, value=7, label="CFG Scale")
                    
                with gr.Row():
                    width = gr.Slider(128, 2048, value=512, step=64, label="Width")
                    height = gr.Slider(128, 2048, value=512, step=64, label="Height")
                
                with gr.Row():
                    use_hires = gr.Checkbox(label="Use HiRes Fix")
                    hires_scale = gr.Slider(1, 4, value=2, label="HiRes Scale")
                    hires_steps = gr.Slider(1, 10, value=3, label="HiRes Creativity Steps")
                
                generate_btn = gr.Button("Generate")
                
            with gr.Column():
                output_image = gr.Image(label="Generated Image")
                used_seed = gr.Number(label="Used Seed", interactive=False)
        
        generate_btn.click(
            fn=generate_image,
            inputs=[prompt, negative_prompt, sampler, steps, seed, cfg_scale,
                   width, height, use_hires, hires_scale, hires_steps],
            outputs=[output_image, used_seed]
        )
    
    return interface

# Ініціалізація та запуск (Initialization and launch)
pipe = load_model()
interface = create_interface()
interface.launch(share=True, debug=True)